In [3]:
from scipy import stats
import numpy as np
import pandas as pd
import os
import plotly.express as px


def vec2str(vec):
    return ''.join([chr(ord('A')+i) for i in vec])


def read_fasta(dir):
    file = open(dir, 'r')
    names = []
    vecs = []
    for line in file.readlines():
        if '#' in line or len(line)<=1:
            continue
        if '>' in line:
            names.append(line.strip('> \n'))
        else:
            vec = [v for v in line.strip(' \n').split(',') if v]
            try:
                vec = [float(s) for s in vec]
            except ValueError:
                vec = [ord(s) for s in vec]
                vec = ''.join([chr(ord('A')+i) for i in vec])
            vecs.append(vec)
    # seqs = {name: vec for name,vec in zip(names,vecs)}
    # return seqs
    return names, vecs

def dense_dists(d):
    num_seqs = d[:,1].max() + 1
    dist = np.zeros((num_seqs, num_seqs))
    for i in range(d.shape[0]):
        dist[d[i,0],d[i,1]] = d[i,2]
    dist = dist + dist.transpose()
    return dist


In [4]:

ds_dir = '/tmp/'
dists_dir = ds_dir+ 'dists/'
sketches_dir = ds_dir + 'sketches/'
seqs_dir = ds_dir + 'seqs.fa'
conf_dir = ds_dir + 'covf.csv'
meth_map= {'mh': 'MH',
           'wmh': 'WMH',
           'omh': 'OMH',
           'ten': 'TenSketch',
           'ten_slide': 'TenSlide',}


names,seqs = read_fasta(seqs_dir)
sketches = dict()
for fname in  os.listdir(sketches_dir):
    names_sk,sk = read_fasta(sketches_dir + fname)
    sketches[fname.split('.')[0]] = np.array(sk)



dists = dict()
for fname in os.listdir(dists_dir):
    df = pd.read_csv(dists_dir + fname)
    dists[fname.split('.')[0]] = df.to_numpy()

print('sketch names : ', sketches.keys())
print('dists names : ', dists.keys())

sketch names :  dict_keys(['ten_slide', 'wmh', 'mh', 'ten', 'omh'])
dists names :  dict_keys(['OMH', 'MH', 'TenSlide', 'TenSketch', 'WMH', 'ED'])


In [6]:
methods = list(dists.keys())
methods.remove('ED')

method = methods[2]
df = {k: d[:,2] for k,d in dists.items()}
df = pd.DataFrame(df)
# fig = px.scatter(df,x='ED', y='OMH', opacity=0.2, marginal_x='histogram', marginal_y='histogram')
d1 = df['ED'].values
d2 = df[method].values
# d1 = d1[d1<1000]
# d2 = d2[d1<1000]
R = stats.spearmanr(d1, d2)
fig = px.scatter(df,x='ED', y=method, opacity=0.8, title='Sp. Corr. = {}'.format(R[0]))
fig.show()




In [5]:
import editdistance
# s1 = seqs[0]
# s2 = seqs[1]
# d = dists['TenSlide']
# r = 20
# i = d[r,0]
# j = d[r,1]
# d = d[r,2]
# ED = editdistance.distance(seqs[i],seqs[j])
# print('exact ED = {}, reported ED = {}'.format(ED, d))
dist = dists['ED']
fig = px.histogram(dist[:,2],nbins=30)
fig.show()

In [17]:
from itertools import product
import os, shutil

def dict_configs(d):
    for k,v in d.items():
        if isinstance(v,list):
            pass
        elif isinstance(v,range):
            d[k] = list(v)
        else:
            d[k] = [v]
    for vcomb in product(*d.values()):
        yield dict(zip(d.keys(), vcomb))

def conf2str(conf, multiline):
    if multiline:
        eol = '\n'
    else:
        eol = ' '
    str = ''
    for k,v in conf.items():
        str += '--{}={}{}'.format(k,v,eol)
    return str

def grid_flagfiles(grid, dir_name):
    for ci, config in enumerate(dict_configs(grid)):
        config['o'] = config['o'] + str(ci) + '/'
        outfile = open('{}/grid_flags_{}'.format(dir_name,ci), 'w')
        if outfile:
            outfile.write(conf2str(config, multiline=True))
        else:
            print('could not open the file')
        outfile.close()

def grid_commands(proj_dir, exec_name, grid, out_dir):
    cmd = 'PROJ_DIR={}\n'.format(proj_dir)
    commands = '{}\necho ">> Project directory ...\\n {}\\n"\n'.format(cmd,cmd)
    proj_dir = '$PROJ_DIR'
    if out_dir.strip()[0] != '/':   # relative output directory
        out_dir = os.path.join(proj_dir, out_dir)
    if exec_name.strip()[0] != '/':
        exec_name = os.path.join(proj_dir, exec_name)
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir)
    for ci, config in enumerate(dict_configs(grid)):
        config['o'] = os.path.join(out_dir, str(ci))
        cmd = '{} {}'.format(exec_name, conf2str(config, multiline=False))
        commands = commands + '\n{}\necho "\\n>> Running Script \\n {}\\n"\n'.format(cmd,cmd)
    outfile = open(os.path.join(out_dir, 'script.sh'), 'w')

    if outfile:
        outfile.write(commands)
    else:
        print('could not open the file')
    outfile.close()


conf_grid = {
    'num_seqs': 1000,
    'alphabet_size': 4,
    'seq_len': 1000,
    'phylogeny_shape': 'path',
    'mutation_type': 'rate',
    'mutation_rate': 1,
    'min_mutation_rate': 0,
    'block_mutation_rate': 0.0,
    'kmer_size': [1,4,8,12],
    'tuple_length': range(2,6),
    'stride': [50,100],
    'window_size': [100,200,500],
    'embed_dim': 30,
    'group_size': 2,
    'hash_alg': 'crc32',
    'num_threads': 0,
}
bin_name = 'experiments'
experiment_dir = '/tmp/test3'
bin_path = '/home/amir/CLionProjects/Project2020-seq-tensor-sketching/cmake-build-release'

# grid_flagfiles(grid=conf_grid, dir_name=experiment_dir)
grid_commands(proj_dir=bin_path, exec_name=bin_name, grid=conf_grid, out_dir=experiment_dir)

In [18]:
from glob import glob

dirs = glob('/tmp/*/*/conf*')
print(dirs)
from pathlib import Path
dirs = list(Path('/tmp/').rglob('conf'))
for path in Path('/tmp/').rglob('conf'):
    print(path)

['/tmp/test2/72/conf', '/tmp/test2/75/conf', '/tmp/test2/81/conf', '/tmp/test2/29/conf', '/tmp/test2/19/conf', '/tmp/test2/79/conf', '/tmp/test2/5/conf', '/tmp/test2/76/conf', '/tmp/test2/63/conf', '/tmp/test2/77/conf', '/tmp/test2/53/conf', '/tmp/test2/46/conf', '/tmp/test2/67/conf', '/tmp/test2/39/conf', '/tmp/test2/11/conf', '/tmp/test2/2/conf', '/tmp/test2/6/conf', '/tmp/test2/20/conf', '/tmp/test2/59/conf', '/tmp/test2/36/conf', '/tmp/test2/18/conf', '/tmp/test2/13/conf', '/tmp/test2/16/conf', '/tmp/test2/4/conf', '/tmp/test2/32/conf', '/tmp/test2/68/conf', '/tmp/test2/31/conf', '/tmp/test2/33/conf', '/tmp/test2/8/conf', '/tmp/test2/51/conf', '/tmp/test2/52/conf', '/tmp/test2/40/conf', '/tmp/test2/64/conf', '/tmp/test2/82/conf', '/tmp/test2/17/conf', '/tmp/test2/27/conf', '/tmp/test2/42/conf', '/tmp/test2/71/conf', '/tmp/test2/58/conf', '/tmp/test2/57/conf', '/tmp/test2/44/conf', '/tmp/test2/30/conf', '/tmp/test2/24/conf', '/tmp/test2/60/conf', '/tmp/test2/47/conf', '/tmp/test2/28

In [15]:
from editdistance import distance as ed
import pandas as pd

df = pd.read_csv('~/PycharmProjects/seqalignment/scratch.csv')
for i in range(len(df)):
    s1 = df.iloc[i,0].strip()
    s2 = df.iloc[i,1].strip()
    ED = df.iloc[i,2]
    print('{},{} -> {} vs {}'.format(ED, ed(s1,s2), s1, s2))

3,3 -> BCCD vs BDDC
3,3 -> BCCD vs CDDA
3,3 -> DBBC vs BDDC
4,4 -> DBBC vs CDDA


In [76]:
path = '/tmp/table1/'
import pandas as pd
from scipy.stats import spearmanr
import os

dists = pd.read_csv(os.path.join(path, 'dists.csv'))
columns = [l for l in dists.columns]
SR = {}
for i in range(2,dists.shape[1]):
    SR[columns[i].strip()] = spearmanr(dists[columns[2]], dists[columns[i]]).correlation

times = pd.read_csv(os.path.join(path, 'times.csv'), skipinitialspace=True)
times = {row['short name'] : row['time'] for _,row in times.iterrows() }
print(SR)
print(times)

flags = pd.read_csv(os.path.join(path, 'flags'), delimiter='=', header=None, names=['name', 'value'])
flags = {row['name'].strip('-'): row['value'] for _,row in flags.iterrows()}
print(flags)

header = """
\begin{table}[h!]
\caption{$1000$ sequence pairs of length $\SLen=20000$ were generated over an alphabet of size $\#\Abc=4$. with the number of random edit operations, uniformly drawn from $\{0,1,\dots,\SLen\}$.  The time column shows normalized time in microseconds, i.e., total time divided by number of sequences,  while the relative time shows the ratio of sketch-based time to the time for computing exact edit distance. As for the the model parameters, embedding dimension is set to $\EDim=30$, and model parameters are (a) MinHash $k = 4$, (b) Weighted MinHash $k=4$, (c) Ordered MinHash $k=3,t=3$, (d) Tensor Sketch $t=3$, (e) Tensor Slide Sketch $w=4000,t=3$.}
\centering
\begin{tabular}{ |c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{\textbf{}} &
\multicolumn{1}{|c|}{\textbf{Correlation}} &
\multicolumn{3}{|c|}{\textbf{AUROC ($\ED \le \cdot $)}} &
\multicolumn{2}{c|}{\textbf{Time}} \\
\hline
"""

body = """
Method  & Spearman  & 0.10 &  0.20 &  0.5 & Abs. ($10^{-3}$ sec) & Rel.(1/ED) \\
\hline
MH & - & 0.500 & 0.500 & 0.500 & \textbf{4.142} & 0.006 \\
\hline
WMH & 0.596 & 0.904 & 0.827 & 0.797 & 55.772 & 0.083 \\
\hline
OMH & 0.668 & 0.965 & 0.933 & 0.821 & 85.444 & 0.127 \\
\hline
TenSketch & 0.966 & \textbf{0.996} & 0.996 & 0.986 & 11.767 & 0.017 \\
\hline
TenSlide & \textbf{0.971} & 0.996 & \textbf{0.996} & \textbf{0.988} & 4.361 & 0.006 \\
\hline
ED & - & - & - & - & 675.420 & 1.000 \\
\hline
"""

footer = """
\end{tabular}
\end{table}"""

table_tex = header + body + footer
# print(table_tex)

{'ED': 1.0, 'MH': nan, 'WMH': 0.372727479363576, 'OMH': 0.5038714344785971, 'TenSketch': 0.6265980206181598, 'TenSlide': 0.7099520215126982}
{'ED': 333.508142, 'MH': 2.158651, nan: 87.251138, 'OMH': 87.254368, 'S2K': 0.079808, 'TenSketch': 34.72476, 'TenSlide': 38.748032, 'WMH': 31.07147}
{'alphabet_size': '4', 'block_mutation_rate': '0', 'embed_dim': '100', 'fix_len': 'true', 'group_size': '2', 'hash_alg': 'crc32', 'kmer_size': '4', 'max_len': '10000', 'max_mutation_rate': '1', 'max_num_blocks': '4', 'min_mutation_rate': '0', 'min_num_blocks': '2', 'mutation_type': 'rate', 'num_bins': '256', 'num_seqs': '5000', 'num_threads': '0', 'o': '/tmp/table1/', 'phylogeny_shape': 'path', 'seq_len': '10000', 'stride': '2000', 'transform': 'none', 'tuple_length': '3', 'window_size': '3000', 'flagfile': '../experiments_flags', 'tab_completion_columns': '80', 'help': 'false', 'helpfull': 'false', 'helppackage': 'false', 'helpshort': 'false', 'helpxml': 'false', 'version': 'false'}


/home/amir/anaconda3/envs/seqalignment/lib/python3.8/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning:

An input array is constant; the correlation coefficent is not defined.



In [77]:
import plotly.express as px
fig = px.scatter(dists, x='ED', y='OMH')
fig.show()